In [1]:
import pandas as pd
from nlpia.data.loaders import get_data
pd.options.display.width=120

INFO:nlpia.constants:Starting logger in nlpia.constants...
INFO:nlpia.loaders:No BIGDATA index found in C:\ProgramData\Anaconda3\lib\site-packages\nlpia\data\bigdata_info.csv so copy C:\ProgramData\Anaconda3\lib\site-packages\nlpia\data\bigdata_info.latest.csv to C:\ProgramData\Anaconda3\lib\site-packages\nlpia\data\bigdata_info.csv if you want to "freeze" it.
INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\ProgramData\\Anaconda3\\lib\\site-packages\\nlpia\\data\\mavis-batey-greetings.csv',), **{'low_memory': False})`...
INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\ProgramData\\Anaconda3\\lib\\site-packages\\nlpia\\data\\sms-spam.csv',), **{'low_memory': False})`...


In [2]:
sms=get_data('sms-spam')

INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\ProgramData\\Anaconda3\\lib\\site-packages\\nlpia\\data\\sms-spam.csv',), **{'nrows': None, 'low_memory': False})`...


In [3]:
index=["spam{}{}".format(i,'!'*j) for (i,j) in zip(range(len(sms)),sms.spam)]

In [4]:
sms.index=index

In [5]:
sms.head()

,spam,text
spam0,0,"Go until jurong point, crazy.. Available only ..."
spam1,0,Ok lar... Joking wif u oni...
spam2!,1,Free entry in 2 a wkly comp to win FA Cup fina...
spam3,0,U dun say so early hor... U c already then say...
spam4,0,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
from nltk.tokenize.casual import casual_tokenize

In [8]:
tfidf=TfidfVectorizer(tokenizer=casual_tokenize)

In [9]:
tfidf_docs=tfidf.fit_transform(raw_documents=sms.text).toarray()

In [10]:
len(tfidf.vocabulary_)

9232

In [11]:
tfidf_docs=pd.DataFrame(tfidf_docs)

In [12]:
tfidf_docs=tfidf_docs-tfidf_docs.mean()

In [13]:
tfidf_docs.shape

(4837, 9232)

In [14]:
sms.spam.sum()

638

In [15]:
from sklearn.decomposition import PCA

In [16]:
pca=PCA(n_components=16)

In [17]:
pca=pca.fit(tfidf_docs)

In [18]:
pca_topic_vectors=pca.transform(tfidf_docs)

In [19]:
columns=['topic{}'.format(i) for i in range(pca.n_components)]

In [20]:
pca_topic_vectors=pd.DataFrame(pca_topic_vectors,columns=columns,index=index)

In [21]:
pca_topic_vectors.head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
spam0,0.201168,0.002759,0.037228,0.010981,-0.019207,-0.052933,0.039108,-0.065877,0.011954,-0.082108,0.007014,0.003900,-0.009920,-0.018896,-0.009127,0.034069
spam1,0.404380,-0.093879,-0.077516,0.050893,0.100062,0.047156,0.022895,0.064805,0.023086,-0.024181,-0.005090,-0.040698,0.030589,-0.037617,0.060921,0.000459
spam2!,-0.030458,-0.048114,0.090183,-0.067078,0.090754,-0.043049,-0.000363,-0.001131,-0.056279,0.051145,0.127465,-0.022684,0.016768,-0.012434,-0.032040,0.051736
spam3,0.329047,-0.032781,-0.034562,-0.015795,0.052273,0.055629,-0.165693,-0.074899,0.061215,-0.107794,0.022755,-0.028829,0.054565,-0.086998,0.035908,-0.061661
spam4,0.002159,0.030850,0.038305,0.033865,-0.074720,-0.092486,-0.043714,0.060733,-0.043994,0.028351,0.029190,0.009121,0.041973,0.019470,-0.075661,-0.006321
spam5!,-0.015985,0.058546,0.014021,-0.006057,0.122312,-0.039897,0.004971,0.167440,-0.022146,0.064539,0.041674,-0.049450,-0.004412,0.095788,-0.005868,-0.064143


In [28]:
list(tfidf.vocabulary_.items())[:10]

[('go', 3807),
 ('until', 8487),
 ('jurong', 4675),
 ('point', 6296),
 (',', 13),
 ('crazy', 2549),
 ('..', 21),
 ('available', 1531),
 ('only', 5910),
 ('in', 4396)]

In [29]:
column_nums,terms=zip(*sorted(zip(tfidf.vocabulary_.values(),tfidf.vocabulary_.keys())))

In [31]:
terms[:10]

('!', '"', '#', '#150', '#5000', '$', '%', '&', "'", '(')

In [32]:
weights=pd.DataFrame(pca.components_,columns=terms,index=['topic{}'.format(i) for i in range(16)])

In [38]:
pd.options.display.max_columns=8

In [39]:
weights.head(4).round(3)

,!,"""",#,#150,...,…,┾,〨ud,鈥
topic0,-0.071,0.008,-0.001,-0.000,...,-0.002,0.001,0.001,0.001
topic1,0.064,0.008,0.000,-0.000,...,0.003,0.001,0.001,0.001
topic2,0.071,0.027,0.000,0.001,...,0.002,-0.001,-0.001,-0.001
topic3,-0.059,-0.032,-0.001,-0.000,...,0.001,0.001,0.001,0.001


In [40]:
pd.options.display.max_columns=12

In [41]:
deals=weights['! ;) :) half off free crazy deal only $ 80 %'.split()].round(3)*100

In [42]:
deals

,!,;),:),half,off,free,crazy,deal,only,$,80,%
topic0,-7.1,0.1,-0.5,-0.0,-0.4,-2.0,-0.0,-0.1,-2.2,0.3,-0.0,-0.0
topic1,6.4,0.0,7.4,0.1,0.4,-2.3,-0.2,-0.1,-3.8,-0.1,-0.0,-0.2
topic2,7.1,0.2,-0.1,0.0,0.3,4.4,0.1,-0.1,0.7,0.0,0.0,0.1
topic3,-5.9,-0.3,-7.1,0.2,0.3,-0.2,0.0,0.1,-2.3,0.1,-0.1,-0.3
topic4,38.1,-0.1,-12.5,-0.1,-0.2,9.9,0.1,-0.2,3.0,0.3,0.1,-0.1
topic5,-26.6,0.1,-1.6,-0.3,-0.7,-1.4,-0.6,-0.2,-1.8,-0.9,0.0,0.0
topic6,-10.9,-0.5,19.9,-0.4,-0.9,-0.6,-0.2,-0.1,-1.4,-0.0,-0.0,-0.1
topic7,16.1,0.1,-17.9,0.8,0.8,-2.8,0.0,0.1,-1.8,-0.3,0.0,-0.1
topic8,34.0,0.1,5.1,-0.5,-0.6,0.1,-0.4,-0.4,3.4,-0.6,-0.0,-0.2
topic9,7.9,-0.3,16.7,1.4,-0.9,6.2,-0.5,-0.4,3.0,-0.5,-0.0,0.0


In [44]:
deals.T.sum()

topic0    -11.9
topic1      7.6
topic2     12.7
topic3    -15.5
topic4     38.3
topic5    -34.0
topic6      4.8
topic7     -5.0
topic8     40.0
topic9     32.6
topic10   -29.1
topic11   -46.5
topic12    31.3
topic13    37.4
topic14    23.5
topic15   -23.9
dtype: float64